<a href="https://colab.research.google.com/github/rcarvajal208/ProyectoMachine/blob/master/P1-CasoFacil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Se importan las librerias

In [0]:
import zipfile
import os 
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from sklearn import preprocessing, model_selection
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

#Se cargan los datos

In [0]:
mundo0 = zipfile.ZipFile("Mundo0.zip", mode="r")
mundo8 = zipfile.ZipFile("Mundo7.zip", mode="r") 
mundo0.extractall(None)
mundo8.extractall(None)

#Se definen las variables 'X' y 'y'

In [28]:
imagenes = []
nivel = []
imagenes0 = mundo0.namelist()
imagenes8 = mundo8.namelist()
#Se agregan las imagenes del mundo 1
i = 1
while(i<=5000):
  filepath = os.path.join("", imagenes0[i]) 
  image = plt.imread(filepath)
  imagenes.append(image)
  nivel.append(0)
  i += 1
#Se agreagan las imagenes del mundo 8
i = 1
while(i<=5000):
  filepath = os.path.join("", imagenes8[i]) 
  image = plt.imread(filepath)
  imagenes.append(image)
  nivel.append(8)
  i += 1
#Se define la salida por categorias, para que se puedan distinguir las clases.
y = to_categorical(np.array(nivel))
X = np.array(imagenes, dtype=np.uint8)  
print('Dimensiones de las variables: ', X.shape, y.shape)

Dimensiones de las variables:  (10000, 200, 240, 3) (10000, 2)


#Preprocesamiento de datos


1.   Se realiza el conjunto de entrenamiento al tomar el 10% de los datos totales.
2.   Se realiza el conjunto de validación al tomar el 10% de los datos de entrenamiento.



In [0]:
#Se definen los porcentajes de entrenamiento y validación
porcentajeEntrenamiento = 0.8
porcentajeValidación = 0.9
#Se determinan los datos de entrenamiento, prueba y validación
x_entrenamiento, x_prueba, y_entrenamiento, y_prueba = model_selection.train_test_split(X,y,test_size=1-porcentajeEntrenamiento)
x_entrenamiento, x_validacion, y_entrenamiento, y_validacion = model_selection.train_test_split(x_entrenamiento,y_entrenamiento,test_size=1-porcentajeValidación)

#Se crea la red neuronal


1.   La red neuronal se realizará a partir de keras
2.   La red va a ser de tipo sequencial, para poder trabajar con diferentes capas, que se encarguen de diferentes funciones.
3.   La primera capa es de tipo Conv2D, esto con el fin de especificar que se esta recibiendo una imagen de dimenciones (200,240,3). Tambien se especifica que su activación es de tipo relu, ya que siempre se recomienda hacer pooling justo despues de haber aplicado una capa con no-linealidad.
4.   Lo siguiente es utilizar MaxPooling2D, para agrupar la presencia de aquellos patrones que se repiten con mayor frecuencia en cada segmento de la imagen.
5.   A continuación se utiliza Dropout, para igualar a 0 un pequeño porcentaje de la información y con esto evitar overfitting en el entrenamiento.
6.   Se utiliza flatten para acoplar los datos que salen de la capa dentreada y ajustarlos para que sirvan como entrada de la capa escondida de clasificación que se utiliza.
7.   Se establece una capa central para para clasificación y se utiliza una activacion tipo relu, ya que sabemos suele entregar los mejores resultados en clasificación.
8.   Nuevamente utilizamos Dropout para evitar overfitting.
9.   Ya para terminar se crea una capa se salida con activación softmax para comprimir los nCentral datos obtenidos por la capa de clasificación, en tan solo nFinal datos que corresponderan al npumero de salidas.
10.  Por ultimo se realiza la compilación de la red, la cual cuenta con:
      1.  Entropia cruzada como función de error, dado que estamos haciendo un clasificador.
      2.  Descenso de gradiente estocastico como algoritmo de optimización.
      3.  La tasa de aprendisaje se fijo en 0.0005, luego de varias pruebas.
      4.  Ademas tambien se indica que debe retornar el porcentaje de aciertos del entrenamiento.


In [0]:
#Función que crea la red neuronal con base en el número de neuronas de cada una de sus capas
def crearRed(nInicial, nCentral, nSalida):
  #Se crea la red sequencial
  RedNeuronal = Sequential()
  #Se crea la capa de entrada
  RedNeuronal.add(Conv2D(nInicial, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(200, 240, 3)))
  #Se realiza una agrupación de datos
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  RedNeuronal.add(MaxPooling2D())
  #Se ajusta los datos para conectarlos a la siguiente capa
  RedNeuronal.add(Dropout(0.5))
  RedNeuronal.add(Flatten())
  #Se crea la capa central de clasificación
  RedNeuronal.add(Dense(nCentral, activation='relu')) 
  RedNeuronal.add(Dropout(0.5)) 
  #Se crea la capa de salida
  RedNeuronal.add(Dense(nSalida, activation='softmax'))
  #Se compila la red
  tasa = 0.0005
  RedNeuronal.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(lr=tasa,decay=tasa/100), metrics=['accuracy'])
  #Se retorna la red creada
  return RedNeuronal

#Validación de número de neuronas


1.   Se van a realizar pruebas de 10 en 10 en cada una de las capas.
2.   Dado que nuestra variable de salida esta restringida a dos posibles salidas debido a la funcion to_categorical(), la ultima capa tambien debera ser de dos salidas, para que la respuesta se pueda comparar con los datos reales.



In [81]:
#Se crea el vector con el número de neuronas
neuronas = [10, 20, 30, 40, 50]
i = 0
#Se recorre la capa de entrada
while(i<len(neuronas)):
  j=0
  #Se recorre la capa central
  while(j<len(neuronas)):
    #Se crea y valida la red neuronal
    RedNeuronal = crearRed(neuronas[i], neuronas[j], 2)
    fit = RedNeuronal.fit(x_validacion, y_validacion, batch_size=64, epochs=6, verbose=0)
    print('[',neuronas[i],',',neuronas[j],', 2 ]:','Tasa de acierto: ',(fit.history['acc']))
    j += 1
  i += 1

[ 10 , 10 , 2 ]: Tasa de acierto:  [0.51, 0.575, 0.57, 0.5725, 0.63625, 0.68375]
[ 10 , 20 , 2 ]: Tasa de acierto:  [0.51875, 0.62375, 0.69, 0.7125, 0.83125, 0.85]
[ 10 , 30 , 2 ]: Tasa de acierto:  [0.49, 0.5525, 0.66875, 0.645, 0.74875, 0.8425]
[ 10 , 40 , 2 ]: Tasa de acierto:  [0.57375, 0.6975, 0.7725, 0.82875, 0.87, 0.8925]
[ 10 , 50 , 2 ]: Tasa de acierto:  [0.62375, 0.8025, 0.8625, 0.88375, 0.915, 0.945]
[ 20 , 10 , 2 ]: Tasa de acierto:  [0.525, 0.635, 0.70125, 0.6575, 0.69875, 0.7175]
[ 20 , 20 , 2 ]: Tasa de acierto:  [0.55125, 0.61125, 0.6375, 0.63125, 0.6425, 0.63875]
[ 20 , 30 , 2 ]: Tasa de acierto:  [0.51875, 0.625, 0.6425, 0.78375, 0.87875, 0.95375]
[ 20 , 40 , 2 ]: Tasa de acierto:  [0.58375, 0.68375, 0.7675, 0.83125, 0.86625, 0.8175]
[ 20 , 50 , 2 ]: Tasa de acierto:  [0.49, 0.6825, 0.79625, 0.84125, 0.93125, 0.93875]
[ 30 , 10 , 2 ]: Tasa de acierto:  [0.57, 0.62125, 0.68375, 0.64375, 0.7125, 0.66125]
[ 30 , 20 , 2 ]: Tasa de acierto:  [0.6575, 0.7825, 0.80875, 0.818

#Se selecciona la red neuronal a crear

1.   Se va a tomar la red con las neuronas [50, 50, 2], dado que se obxervaron muy buenos resultados para este tipo de red.
2.   Se entrena la red con los datos de en trenamiento y esto luego de 5 epochs.
3.   Se guarda la red obtenida.



In [1]:
#Se crea la red neuronal
RedNeuronal = crearRed(50, 50, 2)
#Se muestra como esta compuesta la red
RedNeuronal.summary()
#Se entrena la red
fit = RedNeuronal.fit(x_entrenamiento, y_entrenamiento, batch_size=64, epochs=5, verbose=1)
#Se guarda la red creada
RedNeuronal.save("CasoSencillo.h5py")

NameError: ignored

#Evaluación de la red neuronal
Una vez evaluado el error empirico y rteniendo en cuenta el número de datos \\
Se observa que el indice de confianza es :
##$(1-\delta) = (1 - 2/\exp(|S{\small text}|{ \cdot 2\varepsilon^2 }))$

In [0]:
#Se prueba la red neuronal
prueba = RedNeuronal.evaluate(x_prueba, y_prueba, verbose=0)
#Se calcula el erro empirico, numero de datos y indice de confianza
E_empirico = 1 - [prueba.history['acc']]
N_datos = y_prueba.shape[0]
confianza = 1 - 2/math.exp(N_datos*2*(E_empirico**2))
print('Número de datos de prueba = ' + str(N_datos))
print('Error empirico = ' + str(E_empirico))
print('indice de confianza = ' + str(confianza))